# Scratchwork: experimenting with "words"

In [1]:
import finite_algebras as alg

In [2]:
import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

In [3]:
ex = alg.Examples(alg_dir)

                           Example Algebras
----------------------------------------------------------------------
  16 example algebras are available.
  Use "Examples[INDEX]" to retrieve a specific example,
  where INDEX is the first number on each line below:
----------------------------------------------------------------------
0: A4 -- Alternating group on 4 letters (AKA Tetrahedral group)
1: D3 -- https://en.wikipedia.org/wiki/Dihedral_group_of_order_6
2: D4 -- Dihedral group on four vertices
3: Pinter29 -- Non-abelian group, p.29, 'A Book of Abstract Algebra' by Charles C. Pinter
4: RPS -- Rock, Paper, Scissors Magma
5: S3 -- Symmetric group on 3 letters
6: S3X -- Another version of the symmetric group on 3 letters
7: V4 -- Klein-4 group
8: Z4 -- Cyclic group of order 4
9: F4 -- Field with 4 elements (from Wikipedia)
10: mag_id -- Magma with Identity
11: Example 1.4.1 -- See: Groupoids and Smarandache Groupoids by W. B. Vasantha Kandasamy
12: Ex6 -- Example 6: http://www-groups.m

## Klein-4 Group

In [4]:
v4 = ex[7]
v4

Group(
'V4',
'Klein-4 group',
['e', 'h', 'v', 'r'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
)

In [5]:
v4.about()


** Group **
Name: V4
Instance ID: 140592146921616
Description: Klein-4 group
Order: 4
Identity: e
Commutative? Yes
Cyclic?: No
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       h       h       2
      2       v       v       2
      3       r       r       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]


### Finite Presentation

$V_4 = \langle h, v | h^2 = v^2 = (hv)^2 = e \rangle$

In [6]:
import itertools as it

In [35]:
def cartesian_product(elems):
    '''EXAMPLE: cartesian_product(['h', 'v'])
    ==> ['h_h', 'h_v', 'v_h', 'v_v']'''
    return map(lambda x: '_'.join(x),
               it.product(elems, elems))

def remove_items_from_list(lst, items):
    '''Return the list that results from removing all items in
    items from lst.'''
    if items == []:
        return lst
    else:
        return remove_items_from_list([x for x in lst if x != items[0]],
                                      items[1:])

def reduce_word_by_rel(word, rel):
    rel_ = rel + '_'
    _rel = '_' + rel
    if word == rel:
        return ''
    elif rel_ in word:
        return word.replace(rel_, '')
    elif _rel in word:
        return word.replace(_rel, '')
    else:
        return word
    
def reduce_word_once(word, rels):
    if rels == []:
        return word
    elif rels[0] in word:
        return reduce_word_by_rel(word, rels[0])
    else:
        return reduce_word_once(word, rels[1:])

def reduce_word_recursively(word, rels):
    def reduce_word_aux(old, new):
        if new == old:
            return new
        else:
            return reduce_word_aux(new, reduce_word_once(new, rels))
    return reduce_word_aux('', word)

def multiply_words_OLD(word1, word2, rels):
    prod0 = word1 + '_' + word2
    prod1 = reduce_word_recursively(prod0, rels)
    return prod1

def multiply_words(word1, word2, rels):
    if word1 == '':
        if word2 == '':
            return ''
        else:
            return reduce_word_recursively(word2, rels)
    else:
        if word2 == '':
            return reduce_word_recursively(word1, rels)
        else:
            return reduce_word_recursively(word1 + '_' + word2, rels)

def reduce_words(words, rels):
    return list(set(map(lambda x: reduce_word_recursively(x, rels),
                        words)))

Generators and Relators:

In [36]:
gens = ['h', 'v']
rels = ['h_h', 'v_v', 'h_v_h_v']

In [37]:
cartprod = list(cartesian_product(gens))
cartprod

['h_h', 'h_v', 'v_h', 'v_v']

Remove relations that occur in whole form in the list:

In [38]:
new_items = remove_items_from_list(cartprod, rels)
new_items

['h_v', 'v_h']

Add new items to the generators to create a new list of element candidates:

In [39]:
elems = gens + new_items
elems

['h', 'v', 'h_v', 'v_h']

Now, cross-multiply the new list of element candidates:

In [40]:
foo = list(cartesian_product(elems))
foo

['h_h',
 'h_v',
 'h_h_v',
 'h_v_h',
 'v_h',
 'v_v',
 'v_h_v',
 'v_v_h',
 'h_v_h',
 'h_v_v',
 'h_v_h_v',
 'h_v_v_h',
 'v_h_h',
 'v_h_v',
 'v_h_h_v',
 'v_h_v_h']

In [41]:
foo = remove_items_from_list(foo, elems + rels)
foo

['h_h_v',
 'h_v_h',
 'v_h_v',
 'v_v_h',
 'h_v_h',
 'h_v_v',
 'h_v_v_h',
 'v_h_h',
 'v_h_v',
 'v_h_h_v',
 'v_h_v_h']

In [42]:
foo2 = reduce_words(foo, rels)
foo2

['', 'v_h_v_h', 'h_v_h', 'h', 'v', 'v_h_v']

In [43]:
foo3 = remove_items_from_list(foo2, elems + rels)
foo3

['', 'v_h_v_h', 'h_v_h', 'v_h_v']

In [44]:
for g in gens:
    for r in rels:
        a = multiply_words(g, r, rels)
        b = multiply_words(r, g, rels)
        if a != b:
            if len(a) >= len(b):
                print(f"{a} == {b}")
            else:
                print(f"{b} == {a}")

v_h_v == h
h_v_h == v


In [54]:
multiply_words('h', 'v', rels)

'h_v'

In [55]:
gens

['h', 'v']

In [56]:
pairs = list(it.product(gens, gens))
print(pairs)

fubar = list(set(map(lambda x: multiply_words(*x, rels), pairs)))
fubar.remove('')
print(fubar)

[('h', 'h'), ('h', 'v'), ('v', 'h'), ('v', 'v')]
['h_v', 'v_h']


In [57]:
elems = gens + fubar

pairs = list(it.product(elems, elems))
print(pairs)

fubar = list(set(map(lambda x: multiply_words(*x, rels), pairs)))
fubar.remove('')
print(fubar)

[('h', 'h'), ('h', 'v'), ('h', 'h_v'), ('h', 'v_h'), ('v', 'h'), ('v', 'v'), ('v', 'h_v'), ('v', 'v_h'), ('h_v', 'h'), ('h_v', 'v'), ('h_v', 'h_v'), ('h_v', 'v_h'), ('v_h', 'h'), ('v_h', 'v'), ('v_h', 'h_v'), ('v_h', 'v_h')]
['v_h', 'v_h_v_h', 'h_v', 'h_v_h', 'h', 'v', 'v_h_v']
